# Imports

In [2]:
# IMPORTS
import copy
import numpy as np
from enum import Enum
import bisect 
import time

# CLASS INITIALIZATIONS

In [3]:
class Direction(Enum):
  HORIZONTAL = 1
  VERTICAL = 2

class Coordinate():
  def __init__(self, X, Y):
    self.X = X 
    self.Y = Y
  # Primarily for testing to see if Coordinate works
  def __str__(self):
    return "(" + str(self.X) + ", " + str(self.Y) + ")"

#
# CAR
# 

class Car(object):
  def __init__(self, letter, coordinate, fuel):
    self.letter = letter  # Remember that A is the most important
    self.coordinates = []
    self.coordinates.append(coordinate)
    self.fuel = int(fuel)

  def isA(self):
    return self.letter == 'A'

  def move(self, amt):
    if self.direction == Direction.HORIZONTAL:
      for c in self.coordinates:
        c.Y += amt
    else:
      for c in self.coordinates:
        c.X += amt
    self.fuel -= abs(amt)
  
  def canMove(self):
    return self.fuel > 0

  def compareLetters(self, letter):
    return self.letter == letter

  def addCoordinate(self, newCoord):
    self.coordinates.append(newCoord)
    # Update direction
    # L-R (Horizontal)
    if self.coordinates[0].X == newCoord.X and self.coordinates[0].Y != newCoord.Y:
      self.direction = Direction.HORIZONTAL
    # U-D (Vertical)
    else:
      self.direction = Direction.VERTICAL

  def getLetter(self):
    return self.letter; 

  # Primarily for testing to see if Car works
  def __str__(self):
    s = self.letter + " -> ";
    for i in self.coordinates:
      s += str(i)
    return s + " | " + str(self.fuel) + " | " + str(self.direction)

  @staticmethod
  def create(letter, coordinates, fuel):
    return Car(letter, coordinates, fuel)


#
# BOARD
# 

class Board(object):
  def __init__(self, cars):
    self.grid = [['.' for i in range(6)] for j in range(6)]
    self.size = {'X': 6, 'Y': 6}
    self.cars = []
    for car in cars:  
      c = car
      self.cars.append(copy.deepcopy(c));
      for coord in c.coordinates:
        self.grid[coord.X][coord.Y] = car.letter
        
  def __eq__(self, obj):
        return self.grid == obj.grid

  def findCarPaths(self, car):
    if car.canMove():
      fwd = self.checkPathForward(car)
      bwd = self.checkPathBackward(car)
      carPath = bwd + fwd
    else:
      carPath = []
    return carPath
  
  def moveCar(self, carLetter, amt):
    for car in self.cars: 
      if car.letter == carLetter:
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = "."
        car.move(amt)
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = carLetter

        # Is this a regular car that just left?
        if car.letter == self.grid[2][5] and car.direction == Direction.HORIZONTAL and car.letter != "A":
          for coord in car.coordinates:
            self.grid[coord.X][coord.Y] = "."
          self.cars.remove(car)  
         

  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 4
  # range(5,6) -> 5
  # next gridPos = (1,5)
  # append -> 5 - 4 = 1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 1
  # range(2,6) -> 2, 3, 4, 5
  # next gridPos = (2,3)
  # append -> 2 - 1 = 1
  def checkPathForward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[len(car.coordinates)-1]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint+1, 6):
        #fuel req
      if(car.fuel < i-startPoint):
        continue;
      # Find next position to check    
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i] 
      else:
        gridPos = self.grid[i][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint)
      else:
        break;

    return possibleMoves

  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 3
  # range(3,0,-1) -> 3, 2, 1
  # i = i - 1 -> 2, 1, 0
  # next gridPos = (1,2)
  # append -> 2 - 3 = -1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 0
  # range(0,0) -> NONE
  # next gridPos = NONE
  def checkPathBackward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[0]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint, 0, -1):
      if (car.fuel <= (startPoint-i)):
        continue
      # Find next position to check
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i-1] 
      else:
        gridPos = self.grid[i-1][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint-1)
      else:
        break;

    return possibleMoves

  def winConditionMet(self):
    if self.grid[2][5] == "A":
      return True      
    return False

  # Outputs
  def __str__(self): 
    s = ""
    for x in range(6):
      for y in range(6):
        s += self.grid[x][y]
      s += "\n"
      
    return s

  def stringLine(self):
    s = ""
    for x in range(6):
      for y in range(6):
        s += self.grid[x][y]
    return s

  def AllUsedCars(self):
    s = ""
    for c in cars:
      if(c.fuel < 100):
        s += c.getLetter + " " + c.fuel + " ";
    return s

  #Heuristic definitions:

  #h1 = The number of blocking vehicles
  def h1(self):
        count = 0
        last = " "
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e == last:
                continue
            elif e != ".":
                count += 1
                last = e
        return count
  
  #h2 = The number of blocked positions
  def h2(self):
        count = 0
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e != ".":
                count += 1
        return count
    
  #h3 = The number of blocking vehicles (h1) multiplied by a constant
  def h3(self, constant):
        return self.h1()*constant
    
  #h4 = Is there a car blocking the exit square (3f)
  #Yes: h4 = 1
  #No:  h4 = 0

  def h4(self):
        if self.grid[2][5] != "." and self.grid[2][5] != "A":
            return 1
        else:
            return 0

# Extracting Data From Sample-input.txt

In [4]:
demos = []
demoFuels = []

with open('sample-input-50.txt') as f:
    for line in f:
      if not line.startswith(('#', '\n')):
        in_arr = np.array([line.strip()])
        data = np.char.split(in_arr)[0]
        demos.append(data)
f.close()

print(demos)

[['Z.BOOOZ.B....AAC...D.C...DPPPE.....E'], ['....ZZ....O..AA.OB....OB.....C...DDC'], ['....Z.....Z.AA.BC..O.BC..ODDEE.O....'], ['..ZOOO..ZP.B.AAP.B...P.......C.....C'], ['ZZ..O...BBOC..AAOC...DD...EFF...E...'], ['...O.....O.Z.AAO.Z......B.C...B.CPPP'], ['.....Z..OOOZAA.B.C...B.C.DDPPP......'], ['ZOOO..Z.BC.DAABC.D..PPP.............'], ['ZZBBCC....DE..AADE..O.....OFGG..OF..'], ['ZZ.BCCOOOB.D..EAADFFEGGHIJJ..HI...KK'], ['...ZZ....BCCAA.B.D...E.D...EFF......'], ['OZBBCCOZ.DEEOAADF.GGH.FP..HIIP.JJKKP'], ['OZBCCDOZBEFDOAAEF....GHHII.G..JJPPP.'], ['Z.BOOOZ.BCDDPAACEFPG..EFPGQQQH.....H'], ['...OOO....P...AAP..QQQP...ZBCC..ZB..'], ['Z.O...Z.O...AAO..........BBC.DPPPC.D'], ['..OPPP..OZ.QAAOZBQ...RBQ.CCRD....RD.'], ['ZZ..O.BCDDOEBCAAOE.P.....PF.GG.PFQQQ'], ['..Z.BB..ZC...AACD.....D...E.FF..E...'], ['...OOO......AA.P..ZZ.PB..C.PB..C.QQQ'], ['..ZBCC..ZB....AADE....DE.FFOOO......'], ['.ZZ.BB...CC.AA.OD....OD....OEE......'], ['OOO.P...ZZP...AAP.......BBC.DD..C...'], ['ZBBCDDZ..CEEAA..O..PPPOF..G.OFH

# Setting the boards up

In [ ]:
boards = []
for data in demos:
  # Seperate Grid Data from the Fuel Parameters
  grid = data[0];
  fuels = []
  if len(data) > 1:
    fuels.append(data[1:])

  cars = []
  for row in range(6):
    for col in range(6):
      current = grid[row*6 + col];
      
      # If Unique: Create a car and append
      # If Not Unique: Concat to same letter
      if current == ".":
        continue

      unique = True
      for c in cars:
        if c.compareLetters(current):
          c.addCoordinate(Coordinate(row,col))
          unique = False

      if unique:
        # Now check if there's a unique fuel amount or if its the default 100
        fuelAmt = 100
        if len(fuels) > 0:
          for f in fuels[0]:
            if current == f[0]:
              fuelAmt = f[1:]

        cars.append(Car.create(current, Coordinate(row,col), fuelAmt));

  # Checking if all cars were made properly
  print("CARS: ")
  print("--------")
  for c in cars:
    print(c)
  print()
  
  board = Board(cars)
  # Checking if all boards were made properly
  print("BOARD: ")
  print("--------")
  print(board)
  print()

  boards.append(board)

# Building State Space Search - Uniform Cost Search (UCS)

In [ ]:
class Node(object):
  def __init__(self, board, cost):
    self.board = board
    self.cost = cost
    self.path = []
    self.parent = 0
  def __lt__(self, other):
        return self.cost < other.cost
  def __eq__(self, other):
        return self.board == other.board
  
    
class UCS(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.count = 0

  def recursivePrinting(self, node):
    if node.cost != 0:
      self.recursivePrinting(node.parent)  
    print(node.board)

  def search(self, board):
    self.count += 1
    #Initialise first Node with cost 0 (different for other algorithms)
    a = Node(copy.deepcopy(board), 0)
    self.opened.append(a)
    
    start_time = time.time()
    statessearched = 0
      
    while(1):              
        #If open list is empty, exit
        if(len(self.opened)==0):
            print("No Solution")

            #Write everything to file
            f = open("ucs-sol-"+ str(self.count) +".txt", "w")
            f.write("--------------------------------------------------------------------------------\n")
            f.write("Initial board configuration: " + board.stringLine())
            f.write("\n\n!\n" + str(board) + "\n")
            f.write("Car Fuel Available: ")
            for i in range(len(board.cars)):
                f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                if i < len(board.cars) - 1:
                  f.write(", ")
            f.write("\n Sorry, could not solve the puzzle as specified.\nError: no solution found\n")
            f.write("Runtime: " + str(time.time() - start_time) + " seconds")
            f.write("--------------------------------------------------------------------------------")
            f.close()

            return
        
        #Pop the best candidate node
        curr = self.opened.pop(0)
        
        #If the popped node is the solution, exit out and print stuff (later to file)
        if(curr.board.winConditionMet()):
            print("Initial board configuration")
            print()
            print(board)
            print("Car Fuel Available: ", end = '')
            for i in range(len(board.cars)):
                print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
            print()
            
            print("Runtime: %s seconds" % (time.time() - start_time))
            print("Number of moves: ", curr.cost)
            print("Search path length: %s states" % statessearched)

            print("------")
            #self.recursivePrinting(curr)

            ######

             #Write everything to file
            f = open("ucs-sol-"+ str(self.count) +".txt", "w")
            f.write("--------------------------------------------------------------------------------\n")
            f.write("Initial board configuration: " + board.stringLine())
            f.write("\n\n!\n" + str(board) + "\n")
            f.write("Car Fuel Available: ")
            for i in range(len(board.cars)):
                f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                if i < len(board.cars) - 1:
                  f.write(", ")
            f.write("\n")
            f.write("Runtime: " + str(time.time() - start_time) + " seconds")
            f.write("\nNumber of moves: " + str(curr.cost))
            f.write("\nSearch path length: " + str(statessearched) + " states")

            # Recursive here....


            f.write("\n\n! " + curr.board.AllUsedCars() + "\n")
            f.write(str(curr.board) + "\n")

            f.write("--------------------------------------------------------------------------------")
            f.close()

            return

        #If it's not a solution insert into closed list and increment counter
        self.closed.append(curr)
        statessearched+=1
        #print()
        for c in curr.board.cars:
            paths = curr.board.findCarPaths(c)    
            if len(paths) > 0:  
                #print("Car ", c.letter, " can move:", paths)
                
                for i in range(len(paths)):
                    #Create a new node for successor
                    #Increase the cost of this new node (different in other algorithms)
                    newnode = Node(Board(curr.board.cars), curr.cost+1)   
                    newnode.parent = curr                 
                
                    #Apply the move
                    newnode.board.moveCar(c.letter, paths[i])
                    
                    #If the item is already in the open list, skip it
                    #In other algos, need to check that cost is less
                    #For UCS, this will never be the case, since all costs are 1 in this case (breadth first search)
                    if newnode in self.opened or newnode in self.closed:
                        continue

                    #Insert into open list (in sorted order)
                    bisect.insort(self.opened, newnode)
                    


In [ ]:
currentBoard = Board(boards[46].cars)
ucs = UCS()
ucs.search(currentBoard, 0)

Initial board configuration

....O.
ZB..OC
ZBAAOC
.DPEFF
.DPE..
..P...

Car Fuel Available: O:100, Z:100, B:100, C:100, A:100, D:100, P:100, E:100, F:100, 
Runtime: 53.17274737358093 seconds
Number of moves:  11
Search path length: 3037 states
------
....O.
ZB..OC
ZBAAOC
.DPEFF
.DPE..
..P...

Z...O.
ZB..OC
.BAAOC
.DPEFF
.DPE..
..P...

ZB..O.
ZB..OC
..AAOC
.DPEFF
.DPE..
..P...

ZB..OC
ZB..OC
..AAO.
.DPEFF
.DPE..
..P...

ZB..OC
ZB..OC
AA..O.
.DPEFF
.DPE..
..P...

ZB..OC
ZB..OC
AA..O.
..PEFF
.DPE..
.DP...

ZBP.OC
ZBP.OC
AAP.O.
...EFF
.D.E..
.D....

ZBPEOC
ZBPEOC
AAP.O.
....FF
.D....
.D....

ZBPEOC
ZBPEOC
AAP.O.
FF....
.D....
.D....

ZBPE.C
ZBPE.C
AAP...
FF..O.
.D..O.
.D..O.

ZB.E.C
ZB.E.C
AA....
FFP.O.
.DP.O.
.DP.O.

ZB.E.C
ZB.E.C
....AA
FFP.O.
.DP.O.
.DP.O.



# Building State Space Search - Greedy Best First Search (GBFS)

In [ ]:
class HNode(object):
  def __init__(self, board, cost, parentCost):
    self.board = board
    self.cost = cost
    self.path = []
    self.parent = 0
    self.parentCost = parentCost
  def __lt__(self, other):
        return self.cost < other.cost
  def __eq__(self, other):
        return self.board == other.board
  def addHeuristic(self, heuristic):
        if heuristic == 1:
            self.cost = self.cost + self.board.h1()
        elif heuristic == 2:
            self.cost = self.cost + self.board.h2()
        elif heuristic == 3:
            self.cost = self.cost + self.board.h3(3)
        elif heuristic == 4:
            self.cost = self.cost + self.board.h4()
        return self.cost
  def setCost(self, cost):
        self.cost = cost
        
        
class GBFS(object):
    def __init__(self):
        self.opened = []
        self.closed = []
        self.count = 0
    
    def recursivePrinting(self, node):
        if node.cost != 0:
          self.recursivePrinting(node.parent)  
        print(node.board)
        
    def next(self):
        self.count += 1
        
    def search(self, board, heuristic):
        #Initialise first Node with cost 0
        a = HNode(copy.deepcopy(board), 0, -1)
        self.opened.append(a)
        
        start_time = time.time()
        statessearched = 0
      
        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                #Write everything to file
                f = open("gbfs-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                      f.write(", ")
                f.write("\n Sorry, could not solve the puzzle as specified.\nError: no solution found\n")
                f.write("Runtime: " + str(time.time() - start_time) + " seconds")
                f.write("--------------------------------------------------------------------------------")
                f.close()
                return
        
            #Pop the best candidate node
            curr = self.opened.pop(0)
            curr.setCost(curr.parentCost + 1)
        
            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()
            
                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                #self.recursivePrinting(curr)

                ######

                #Write everything to file
                f = open("gbfs-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                      f.write(", ")
                f.write("\n")
                f.write("Runtime: " + str(time.time() - start_time) + " seconds")
                f.write("\nNumber of moves: " + str(curr.cost))
                f.write("\nSearch path length: " + str(statessearched) + " states")

                # Recursive here....


                f.write("\n\n! " + curr.board.AllUsedCars() + "\n")
                f.write(str(curr.board) + "\n")

                f.write("--------------------------------------------------------------------------------")
                f.close()

                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
        
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)
                    
                    for i in range(len(paths)):
                        #Create a new node for successor
                        newnode = HNode(Board(curr.board.cars), 0, curr.cost)   
                        newnode.parent = curr                 
                
                        #Apply the move
                        newnode.board.moveCar(c.letter, paths[i])
                        newnode.addHeuristic(heuristic)
                    
                        #If the node is already in the open or closed list, skip it
                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        bisect.insort(self.opened, newnode)

In [ ]:
currentBoard = Board(boards[46].cars)
gbfs = GBFS()
gbfs.search(currentBoard, 2)

Initial board configuration

....O.
ZB..OC
ZBAAOC
.DPEFF
.DPE..
..P...

Car Fuel Available: O:100, Z:100, B:100, C:100, A:100, D:100, P:100, E:100, F:100, 
Runtime: 12.705652475357056 seconds
Number of moves:  12
Search path length: 1217 states
------
....O.
ZB..OC
ZBAAOC
.DPEFF
.DPE..
..P...

....OC
ZB..OC
ZBAAO.
.DPEFF
.DPE..
..P...

Z...OC
ZB..OC
.BAAO.
.DPEFF
.DPE..
..P...

ZB..OC
ZB..OC
..AAO.
.DPEFF
.DPE..
..P...

ZB..OC
ZB..OC
AA..O.
.DPEFF
.DPE..
..P...

ZB.EOC
ZB.EOC
AA..O.
.DP.FF
.DP...
..P...

ZBPEOC
ZBPEOC
AAP.O.
.D..FF
.D....
......

ZBPEOC
ZBPEOC
AAP.O.
.DFF..
.D....
......

ZBPE.C
ZBPE.C
AAP...
.DFFO.
.D..O.
....O.

ZBPE.C
ZBPE.C
AAP...
..FFO.
.D..O.
.D..O.

ZBPE.C
ZBPE.C
AAP...
FF..O.
.D..O.
.D..O.

ZB.E.C
ZB.E.C
AA....
FFP.O.
.DP.O.
.DP.O.

ZB.E.C
ZB.E.C
....AA
FFP.O.
.DP.O.
.DP.O.



# Building State Space Search - Algorithm A or A* (A/A*)

In [ ]:
class AAStar(object):
    def __init__(self):
        self.opened = []
        self.closed = []
        self.count = 0
    
    def recursivePrinting(self, node):
        if node.cost != 0:
          self.recursivePrinting(node.parent)  
        print(node.board)
        
    def next(self):
        self.count += 1
        
    def search(self, board, heuristic):
        #Initialise first Node with cost 0
        a = HNode(copy.deepcopy(board), 0, -1)
        
        self.opened.append(a)
        
        start_time = time.time()
        statessearched = 0
      
        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                #Write everything to file
                f = open("a-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                      f.write(", ")
                f.write("\n Sorry, could not solve the puzzle as specified.\nError: no solution found\n")
                f.write("Runtime: " + str(time.time() - start_time) + " seconds")
                f.write("--------------------------------------------------------------------------------")
                f.close()
                return
        
            #Pop the best candidate node
            curr = self.opened.pop(0)
            curr.setCost(curr.parentCost + 1)
        
            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()
            
                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                #self.recursivePrinting(curr)

                #Write everything to file
                f = open("a-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                      f.write(", ")
                f.write("\n")
                f.write("Runtime: " + str(time.time() - start_time) + " seconds")
                f.write("\nNumber of moves: " + str(curr.cost))
                f.write("\nSearch path length: " + str(statessearched) + " states")

                # Recursive here....


                f.write("\n\n! " + curr.board.AllUsedCars() + "\n")
                f.write(str(curr.board) + "\n")

                f.write("--------------------------------------------------------------------------------")
                f.close()

                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
        
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)
                    
                    for i in range(len(paths)):
                        #Create a new node for successor
                        newnode = HNode(Board(curr.board.cars), curr.cost+1, curr.cost)   
                        newnode.parent = curr                 
                
                        #Apply the move
                        newnode.board.moveCar(c.letter, paths[i])
                        newnode.addHeuristic(heuristic)
                    
                        #If the node is already in the open or closed list, skip if the cost in is higher, else replace
                        if newnode in self.opened:
                            for n in self.opened:
                                if n == newnode and n.cost > newnode.cost:
                                    self.opened.remove(n)
                                    bisect.insort(self.opened, newnode)
                                    break
                            continue
                        elif newnode in self.closed:
                            
                            continue
                        else:
                            #Insert into open list (in sorted order)
                            bisect.insort(self.opened, newnode)

In [8]:
currentBoard = Board(boards[46].cars)
aastar = AAStar()
aastar.search(currentBoard, 2)

Initial board configuration

....O.
ZB..OC
ZBAAOC
.DPEFF
.DPE..
..P...

Car Fuel Available: O:100, Z:100, B:100, C:100, A:100, D:100, P:100, E:100, F:100, 
Runtime: 16.722499132156372 seconds
Number of moves:  11
Search path length: 2274 states
------
....O.
ZB..OC
ZBAAOC
.DPEFF
.DPE..
..P...

....OC
ZB..OC
ZBAAO.
.DPEFF
.DPE..
..P...

Z...OC
ZB..OC
.BAAO.
.DPEFF
.DPE..
..P...

ZB..OC
ZB..OC
..AAO.
.DPEFF
.DPE..
..P...

ZB..OC
ZB..OC
AA..O.
.DPEFF
.DPE..
..P...

ZB..OC
ZB..OC
AA..O.
..PEFF
.DPE..
.DP...

ZB.EOC
ZB.EOC
AA..O.
..P.FF
.DP...
.DP...

ZBPEOC
ZBPEOC
AAP.O.
....FF
.D....
.D....

ZBPEOC
ZBPEOC
AAP.O.
FF....
.D....
.D....

ZBPE.C
ZBPE.C
AAP...
FF..O.
.D..O.
.D..O.

ZB.E.C
ZB.E.C
AA....
FFP.O.
.DP.O.
.DP.O.

ZB.E.C
ZB.E.C
....AA
FFP.O.
.DP.O.
.DP.O.



# Output to File

In [ ]:
ucs = UCS()

for b in boards:
    currentBoard = Board(b.cars) 
    ucs.search(currentBoard)

In [ ]:
gbfs = GBFS()

for b in boards:
    currentBoard = Board(b.cars) 
    gbfs.next()
    for i in range(1,5): 
        gbfs.search(currentBoard, i)
  

In [13]:
aastar = AAStar()

for b in boards:
    currentBoard = Board(b.cars) 
    aastar.next()
    for i in range(1,5): 
        aastar.search(currentBoard, i)

Initial board configuration

Z.BOOO
Z.B...
.AAC..
.D.C..
.DPPPE
.....E

Car Fuel Available: Z:100, B:100, O:100, A:100, C:100, D:100, P:100, E:100, 
Runtime: 5.570498943328857 seconds
Number of moves:  9
Search path length: 1199 states
------
Z.BOOO
Z.B...
.AAC..
.D.C..
.DPPPE
.....E

..BOOO
..B...
.AAC..
ZD.C..
ZDPPPE
.....E

..BOOO
..B...
AA.C..
ZD.C..
ZDPPPE
.....E

...OOO
..B...
AABC..
ZD.C..
ZDPPPE
.....E

OOO...
..B...
AABC..
ZD.C..
ZDPPPE
.....E

OOOC..
..BC..
AAB...
ZD....
ZDPPPE
.....E

OOOC.E
..BC.E
AAB...
ZD....
ZDPPP.
......

OOOC.E
..BC.E
AAB...
ZD....
ZD.PPP
......

OOOC.E
...C.E
AA....
ZDB...
ZDBPPP
......

OOOC.E
...C.E
....AA
ZDB...
ZDBPPP
......

Initial board configuration

Z.BOOO
Z.B...
.AAC..
.D.C..
.DPPPE
.....E

Car Fuel Available: Z:100, B:100, O:100, A:100, C:100, D:100, P:100, E:100, 
Runtime: 0.0305025577545166 seconds
Number of moves:  9
Search path length: 4 states
------
Z.BOOO
Z.B...
.AAC..
.D.C..
.DPPPE
.....E

..BOOO
..B...
.AAC..
ZD.C..
ZDPPPE
.....E

